In [1]:
from __future__ import absolute_import
from google.cloud import language
import google.cloud
import pandas as pd
import numpy as np
import tqdm
import operator
import facebook
import json
import tweepy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import datetime


#Create client instance for Google Natural Language API
client = language.Client()

In [9]:
all_tweets_df = pd.DataFrame.from_csv('/Users/thays/Desktop/RDU_pitch/RDUpitch_ALL_tweets.csv', encoding='utf-8')
all_tweets_df.reset_index(inplace=True, drop = True)
all_tweets_df

,date,time,location,handle,brand,tweet,posting_user,sentiment,magnitude,entities,entity_salience,geolocation,place,coordinates
0,2017-03-07,15:56:35,NaN,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Townsend,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-03-07,15:51:24,Greencastle IN,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Mark Scott,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-07,15:49:47,"Terre Haute, IN",@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Holly Fields✨,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-03-07,15:48:12,,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Mary,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-07,15:41:42,NaN,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,tweetybird🐥,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017-03-07,15:40:55,indianapolis,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Joe,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2017-03-07,15:40:32,"Pendleton, IN",@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Jill McKinney,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017-03-07,15:39:28,Indy,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Edward Holloran III,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017-03-07,15:38:24,NaN,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Amanda Wood,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2017-03-07,15:35:07,"Fort Atkinson, WI",@INDairport,Indianapolis,#Indianapolis #Airport Welcomes New @FlyFronti...,Airport Business,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
all_tweets_df[['brand', 'date']].groupby('brand').count()

,date
brand,
Atlanta,680
Austin,276
Charlotte,554
Indianapolis,256
Louisville,26
Nashville,120
Portland,322
RDU,242
San_Diego,180


In [12]:
# Sentiment Analysis of Replies

sentiments_list = []
magnitudes_list = []
entities_name_list = []
entities_salience_list = []
comments = list(all_tweets_df['tweet'])

for comment in tqdm.tqdm(comments):

    try:
        document = client.document_from_text(comment)
        sentiment = document.analyze_sentiment()
        entities = document.analyze_entities()
        sentiment_score = sentiment.score
        magnitude = sentiment.magnitude
        entities_name = [str(x.name) for x in entities[:3]]
        entities_salience = [x.salience for x in entities[:3]]
    except UnicodeEncodeError:
        sentiment_score = None
        magnitude = [None]
        entities_name = [None]
        entities_salience = [None]


    sentiments_list.append(sentiment_score)
    magnitudes_list.append(magnitude)
    entities_name_list.append(entities_name)
    entities_salience_list.append(entities_salience)
    
    time.sleep(.2)
        
all_tweets_df['sentiment'] = sentiments_list
all_tweets_df['magnitude'] = magnitudes_list
all_tweets_df['entities'] = entities_name_list
all_tweets_df['entity_salience'] = entities_salience_list


100%|██████████| 2656/2656 [15:38<00:00,  2.87it/s]


In [13]:
all_tweets_df.to_csv('/Users/thays/Desktop/RDU_pitch/RDUpitch_ALL_tweets_sentiment.csv', encoding='utf-8')

In [14]:
all_tweets_df

,date,time,location,handle,brand,tweet,posting_user,sentiment,magnitude,entities,entity_salience,geolocation,place,coordinates
0,2017-03-07,15:56:35,NaN,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Townsend,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
1,2017-03-07,15:51:24,Greencastle IN,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Mark Scott,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
2,2017-03-07,15:49:47,"Terre Haute, IN",@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Holly Fields✨,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
3,2017-03-07,15:48:12,,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Mary,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
4,2017-03-07,15:41:42,NaN,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,tweetybird🐥,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
5,2017-03-07,15:40:55,indianapolis,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Joe,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
6,2017-03-07,15:40:32,"Pendleton, IN",@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Jill McKinney,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
7,2017-03-07,15:39:28,Indy,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Edward Holloran III,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
8,2017-03-07,15:38:24,NaN,@INDairport,Indianapolis,RT @joetamborello: People really like @INDAirp...,Amanda Wood,0.4,1.3,"[People, RT @joetamborello, @INDAirport]","[0.3658298, 0.2567241, 0.12963061]",NaN,NaN,NaN
9,2017-03-07,15:35:07,"Fort Atkinson, WI",@INDairport,Indianapolis,#Indianapolis #Airport Welcomes New @FlyFronti...,Airport Business,0.3,0.3,"[Indianapolis #Airport, New @FlyFrontier Fligh...","[0.40594935, 0.2823747, 0.21015264]",NaN,NaN,NaN
